# Getting a simple dialect to work

In this example, we show how to use the xDSL core infrastructure to define some simple abstractions for a database compiler. Furthermore, we show uses of the rewrite engine to rewrite the data structures.


As a first step, we decide to work on bag-like structures, so we decide to implement an Attribute corresponding to a bag type.

In [67]:
from xdsl.ir import ParametrizedAttribute
from xdsl.irdl import irdl_attr_definition, ParameterDef, AnyAttr

@irdl_attr_definition
class Bag(ParametrizedAttribute):
  name = "sql.bag"
  schema: ParameterDef[AnyAttr()]

Instantiating this Attribute with an i32, and then printing it, looks like the following:

In [68]:
from xdsl.printer import Printer
from xdsl.dialects.builtin import i32

printer = Printer()
printer.print_attribute(Bag([i32]))

!sql.bag<!i32>

Secondly, we define an abstraction modeling a table.

In [69]:
from xdsl.ir import Operation
from xdsl.irdl import irdl_op_definition, AttributeDef, OpResult
from xdsl.dialects.builtin import StringAttr
from typing import Annotated


@irdl_op_definition
class Table(Operation):
  name = "sql.table"
  table_name = AttributeDef(StringAttr)
  result_bag: Annotated[OpResult, Bag]

Using this operation, we can create the first full query:

```sql
SELECT * from T
```

In [70]:
t = Table.build(attributes={"table_name":StringAttr.from_str("T")}, result_types=[Bag([(i32)])])
printer.print_op(t)

%0 : !sql.bag<!i32> = sql.table() ["table_name" = "T"]


In xDSL, all IRs need a ModuleOp as the outermost Operation.

In [71]:
from xdsl.dialects.builtin import ModuleOp
from xdsl.ir import Region

m = ModuleOp.from_region_or_ops([t])
printer.print(m)

builtin.module() {
  %0 : !sql.bag<!i32> = sql.table() ["table_name" = "T"]
}


In this example, the SSAValue %0 is just flying around. We want to make sure it is actually bound to somewhere, such that we now what to do with it during compilation. Therefore, we introduce a SinkOp, which returns the data in the bag to the executor of the Query.

In [72]:
from xdsl.irdl import Operand

@irdl_op_definition
class SinkOp(Operation):
  name = "sql.sink"
  bag: Annotated[Operand, Bag]

In [73]:
m.body.blocks[0].add_op(SinkOp.build(operands=[t]))

In [74]:
printer.print(m)

builtin.module() {
  %0 : !sql.bag<!i32> = sql.table() ["table_name" = "T"]
  sql.sink(%0 : !sql.bag<!i32>)
}


Obviously, we want to do more involved stuff. Say we would like to abstract something like:

```SQL
SELECT * from T 
WHERE T.a < 5
```

In order to abstract this, we need an abstraction for selections, which we decided to make an operation. The actual condition to filter becomes a nested region. Additionally, we decide to reuse dialects already defined within xDSL, in particular the arith dialect.

In [75]:
from xdsl.irdl import RegionDef

@irdl_op_definition
class Selection(Operation):
  name = "sql.selection"
  input_bag: Annotated[Operand, Bag]
  filter = RegionDef()
  result_bag: Annotated[OpResult, Bag]

We instantiate this in two steps. First, we build the filter region:

In [76]:
from xdsl.ir import Block
from xdsl.dialects.arith import Constant, Cmpi
from xdsl.dialects.scf import Yield

filter = Region.from_block_list([Block.from_callable(
    [i32], # filter argument
    lambda arg: [
        const := Constant.from_int_and_width(5, 32),
        cmp := Cmpi.from_mnemonic(arg, const, "sgt"),
        Yield.get(cmp)
    ])])

In [77]:
sel = Selection.build(
    result_types=[Bag([i32])],
    operands=[t],
    regions=[filter])

printer.print_op(sel)

%1 : !sql.bag<!i32> = sql.selection(%0 : !sql.bag<!i32>) {
^0(%2 : !i32):
  %3 : !i32 = arith.constant() ["value" = 5 : !i32]
  %4 : !i1 = arith.cmpi(%2 : !i32, %3 : !i32) ["predicate" = 4 : !i64]
  scf.yield(%4 : !i1)
}
